#### ROUTING GAME LIBRARY

In [12]:
using Revise, JLD2
include("routing_games.jl")

plot_labeled (generic function with 1 method)

#### FORWARD SOLVER: game(p,E,s, b, C) ==> (x, v)

In [6]:
# include("routing_solvers/routing_solver_entropy_jump.jl")
include("routing_solvers/routing_solver_entropy_nlsolve.jl")
include("routing_solvers/routing_solver_exact_jump.jl")

solve_routing (generic function with 1 method)

##### test forward solvers

In [ ]:
pa = grid_graph3_4_players_reduced()
b = 0.1*ones(pa.p*pa.m)
C = zeros(pa.p*pa.m, pa.p*pa.m)
x_init = rand(pa.p*pa.m)
v_init = zeros(pa.p*(pa.n-1))

x, v = solve_entropy_routing_jump(pa, b, C, 0.01, x_init, v_init)
x, v = solve_entropy_routing_nlsolve(pa, b, C, 0.01, x_init, v_init)
x, v = solve_routing(pa, b, C, x_init, v_init)

#### BACKWARD SOLVER: (p,E,s, x_hat) ==> (b, C)

In [10]:
include("approx_proj_gradient.jl")

approx_proj_grad

##### test backward solver

In [ ]:
using ArgParse
using JLD2
using Plots, GraphPlot, ColorSchemes

# parse cmd args
function parse_commandline()
    s = ArgParseSettings()

    @add_arg_table s begin
        "--rho"
            help = "rho"
            arg_type = Float64
            default = 0.5
        "--lambda"
            help = "entropy reg term"
            arg_type = Float64
            default = 0.01
        "--alpha"
            help = "GD step size"
            arg_type = Float64
            default = 0.005
        "--epsilon"
            help = "convergence threshold"
            arg_type = Float64
            default = 1e-3
        "--max_iter"
            help = "maximum iter allowed"
            arg_type = Int64
            default = 10
        "--plot"
            help = "plot exp result or not"
            arg_type = Bool
            default = true
        "--forward_solver"
            help = "nlsolve or ipopt"
            arg_type = String
            default = "nlsolve"
    end

    return parse_args(s)
end
args = parse_commandline()

In [ ]:
pa = grid_graph3_4_players_reduced()

x = zeros(pa.p*pa.m)
v = zeros(pa.p*(pa.n-1))
b = 0.1*ones(pa.p*pa.m)
C = zeros(pa.p*pa.m, pa.p*pa.m)

# assign parameters
λ = 0.01
α = 0.005
ϵ = 1e-3
ρ = 0.5
max_iter = 10

println("----------- $(pa.game_name)_λ=($λ)_α=($α)_ϵ=($ϵ)_ρ=($ρ) -----------")
x, v, b, C, lambda_vals, ψ_vals_exact = approx_proj_grad(pa, λ, α, ϵ, ρ, max_iter, x, v, b, C)

#### Rho choice experiment

In [20]:
using Plots, GraphPlot, Colors
using Graphs, GraphRecipes
using ArgParse
using JLD2
using IterativeSolvers

# helper function: projection onto B
function proj_B(b)
    return min.(0.1, max.(0, b))
end

# helper function: projection onto D
function proj_D(C, ρ, pa)
    C1 = 0.5 * (C - C')
    for n in 1:pa.p
        C1[pa.m*(n-1)+1:pa.m*n, pa.m*(n-1)+1:pa.m*n] = zeros(pa.m, pa.m)
    end
    
    C2 = 0.5 * (C + C')
    s = eigvals(C2)
    U = eigvecs(C2)
    
    A = real(C1 + U * diagm(vec(max.(s,zeros(length(s))))) * U')
    return ρ / max.(ρ, norm(A)) * A
end

"""
Approximate projected gradient method (measuring function ψ(x)=1/2*||x-x̂||^2)
Inputs:
- p, number of players
- E, incidence matrix
- s, source-sink vector
- x̂, desired Nash solution
- λ, entropy weight
- α, step size
- ϵ, stopping tolerance
- ρ, modification allowance
- max_iter

Returns:
- x: mixed eq strategies for each player on space of links
- b
- C
"""

function approx_proj_grad(pa, λ, α, ϵ, ρ, max_iter, x_init, v_init, b_init, C_init)
   
    # initiate b, b_plus, C, C_plus
    b = zeros(pa.p*pa.m)
    b_plus = b_init
    C = 0.1*I(pa.p*pa.m)
    C_plus = C_init

    # initiate x, v, x_exact, v_exact
    x = x_init
    v = v_init

    # data output placeholders
    lambda_vals = Float64[]
    ψ_vals = Float64[]

    for i in 1:max_iter
        if max(norm(b-b_plus), norm(C-C_plus)) <= ϵ || 0.5 * norm(x-pa.x̂)^2 <= ϵ
        # if max(norm(b-b_plus), norm(C-C_plus)) <= ϵ
        # if 0.5 * norm(x-x̂)^2 <= ϵ
            println("Converged.")
            break
        elseif i == max_iter
            println("Reached max_iter of $max_iter, break.")
            break
        else
            println("\t\t### ITER $i ###\n")
            
            # update b, C
            println("\t\t\tupdate b, C...")
            b = b_plus
            C = C_plus

            println("\t\t\t*** Compute (b, C) => (x, v) => ψ(x) for iter $i ***")
            x, v = solve_entropy_routing_nlsolve(pa, b, C, λ, x, v)
            push!(ψ_vals, 0.5 * norm(x-pa.x̂)^2)

            # record current λ value
            push!(lambda_vals, λ)
            
            # compute D, J 
            println("\t\t\tcompute D, J...")
            D = diagm(vec(exp.(1/λ * (pa.E_diag'*v-b-C*x) - ones(pa.p*pa.m, 1)))) # dim: 18x18
            J = [I(pa.p*pa.m)+1/λ*D*C     1/λ*D*pa.E_diag'; -pa.E_diag      zeros(pa.p*(pa.n-1), pa.p*(pa.n-1))] # dim: 27x27

            # compute ∇ψ_x
            println("\t\t\tcompute ∇ψ_x...")
            ∇ψ_x = x - pa.x̂ # 18x1

            # compute ∇̂ψ_b, ∇̂ψ_C
            println("\t\t\tcompute ∇̂ψ_b, ∇ψ_C...")
            ∇̂ψ_b = - 1/λ * [D' zeros(pa.p*pa.m, pa.p*(pa.n-1))] * pinv(J)' * [∇ψ_x; zeros(pa.p*(pa.n-1))]
            ∇̂ψ_C = - 1/λ * [D' zeros(pa.p*pa.m, pa.p*(pa.n-1))] * pinv(J)' * [∇ψ_x; zeros(pa.p*(pa.n-1))] * x' # 18x1

            # update b_plus, C_plus
            println("\t\t\tcompute b_plus, C_plus...\n")
            b_plus = proj_B(b - α * ∇̂ψ_b)
            C_plus = proj_D(C - α * ∇̂ψ_C, ρ, pa)

        end
    end

    (;x = x,
      v = v,
      b = b,
      C = C,
      ψ_vals = ψ_vals,
      lambda_vals = lambda_vals
)

end

approx_proj_grad (generic function with 1 method)

In [25]:
pa = grid_graph3_2_players_reduced()

x = zeros(pa.p*pa.m)
v = zeros(pa.p*(pa.n-1))
b = 0.1*ones(pa.p*pa.m)
C = zeros(pa.p*pa.m, pa.p*pa.m)

# assign parameters
λ = 0.01
α = 0.01
ϵ = 0.01
ρ = 0.1
max_iter = 10

# calling method
println("----------- $(pa.game_name)_λ=($λ)_α=($α)_ϵ=($ϵ)_ρ=($ρ) -----------")
x, v, b, C, lambda_vals, ψ_vals = approx_proj_grad(pa, λ, α, ϵ, ρ, max_iter, x, v, b, C)

""" SAVING RESULT """
dir = "rho_exp/results/$(pa.game_name)/$(pa.game_name)_λ=($λ)_α=($α)_ϵ=($ϵ)_ρ=($ρ)"; mkpath(dir) # mkdir if not exists
println("saving output to '$dir'")
@save "$dir/output.jld2" x v b C lambda_vals ψ_vals

plot(ψ_vals, xlabel="iter", ylabel="ψ(x)"); savefig("$dir/ψ_vals_plots_λ=($λ)_α=($α)_ϵ=($ϵ)_ρ=($ρ).png")
@show ψ_vals

----------- grid_graph3_2_players_reduced_λ=(0.01)_α=(0.01)_ϵ=(0.01)_ρ=(0.1) -----------
		### ITER 1 ###

			update b, C...
			*** Compute (b, C) => (x, v) => ψ(x) for iter 1 ***
			--- entropy routing solver (NLsolve) ---
			compute D, J...
			compute ∇ψ_x...
			compute ∇̂ψ_b, ∇ψ_C...
			compute b_plus, C_plus...

		### ITER 2 ###

			update b, C...
			*** Compute (b, C) => (x, v) => ψ(x) for iter 2 ***
			--- entropy routing solver (NLsolve) ---
			compute D, J...
			compute ∇ψ_x...
			compute ∇̂ψ_b, ∇ψ_C...
			compute b_plus, C_plus...

		### ITER 3 ###

			update b, C...
			*** Compute (b, C) => (x, v) => ψ(x) for iter 3 ***
			--- entropy routing solver (NLsolve) ---
			compute D, J...
			compute ∇ψ_x...
			compute ∇̂ψ_b, ∇ψ_C...
			compute b_plus, C_plus...

Converged.
saving output to 'rho_exp/results/grid_graph3_2_players_reduced/grid_graph3_2_players_reduced_λ=(0.01)_α=(0.01)_ϵ=(0.01)_ρ=(0.1)'
ψ_vals = [0.01, 0.01, 0.01]


3-element Vector{Float64}:
 0.01
 0.01
 0.01

#### Homotopy Experiment

In [ ]:
function homotopy_exp_parameter_choice(pa, α, ϵ, ρ, max_iter, λ_list)

    # output placeholder
    x = zeros(pa.p*pa.m)
    v = zeros(pa.p*(pa.n-1))
    b = 0.1*ones(pa.p*pa.m)
    C = zeros(pa.p*pa.m, pa.p*pa.m)
    lambda_vals_list = Vector{Float64}[]
    ψ_vals_exact_list = Float64[]
    ψ_x_exact = Inf

    # run experiment
    println("----------- $(pa.game_name)_ρ=($ρ)_λ_list=($λ_list)_α=($α)_ϵ=($ϵ) -----------\n")
    for λ in λ_list
        if ψ_x_exact <= ϵ
            println("\t----- approx_proj_grad converged -----")
            break
        else
            println("\t----- approx_proj_grad with {λ=$λ} -----\n")
            x, v, b, C, lambda_vals, ψ_vals_exact = approx_proj_grad(pa, λ, α, ϵ, ρ, max_iter, x, v, b, C)
            push!(lambda_vals_list, lambda_vals)
            append!(ψ_vals_exact_list, ψ_vals_exact)
            ψ_x_exact = ψ_vals_exact_list[end]
        end
    end
    println("--------------------------------\n")

     # create an individual folder under homotopy_results/
    dir = "homotopy_results/$(pa.game_name)"
    mkpath(dir) # mkdir if not exists

    # save data to folder
    println("saving result...")
    @save "$dir/$(pa.game_name)_λ_list=($λ_list)_homotopy.jld2" ρ λ_list α ϵ max_iter x v b C lambda_vals_list ψ_vals_exact_list
    println("saved result to '$dir/$(pa.game_name)_λ_list=($λ_list)_homotopy.jld2'")
    println("--------------------------------\n")

    (;x = x,
      v = v,
      b = b,
      C = C,
      lambda_vals_list = lambda_vals_list,
      ψ_vals_exact_list = ψ_vals_exact_list,
      dir = dir)
end

In [ ]:
""" RUNNING EXP: PARAMETER CHOICE """
# assign parameters
α = args["alpha"]
ϵ = args["epsilon"]
ρ = args["rho"]
max_iter = args["max_iter"]
λ_list = [1.0, 0.5, 0.1, 0.02]

# calling method
x, v, b, C, lambda_vals_list, ψ_vals_exact_list, dir = homotopy_exp_parameter_choice(grid_graph3_4_players_reduced(), α, ϵ, ρ, max_iter, λ_list)

In [ ]:
using IJulia

pa = grid_graph3_4_players_reduced()

@load "$dir/$(pa.game_name)_λ_list=($λ_list)_homotopy.jld2" ρ λ_list α ϵ max_iter x v b C lambda_vals_list ψ_vals_exact_list

In [ ]:
# plot single axis (log-scaled)
println("plotting homotopy...")
p1 = plot(title="ρ=($ρ), α=($α) \n λ_list=($λ_list)")
lambda_vals_list_flattened = collect(Iterators.flatten(lambda_vals_list))
lambda_colors = palette([:green, :blue], length(lambda_vals_list))
for i in eachindex(lambda_vals_list)
    lambda_vals = lambda_vals_list[i]
    plot!(findall(x->x==lambda_vals[1], lambda_vals_list_flattened), lambda_vals, yscale=:log10, color = lambda_colors[i], marker=:circle, label="λ=$(lambda_vals[1])", leg=:bottomleft)
end 
plot!(ψ_vals_exact_list, yscale=:log10, c=:red, linestyle=:dash, label="ψ(x)_exact", leg=:bottomleft)
hline!([ϵ], label="ϵ=$ϵ", c=:grey)
savefig("$dir/$(pa.game_name)_λ_list=($λ_list)_homotopy_log.png")
println("saved to '$dir/$(pa.game_name)_λ_list=($λ_list)_homotopy_log.png'")
IJulia.display(p1)

# plot single axis (linear-scaled)
p2 = plot(title="ρ=($ρ), α=($α) \n λ_list=($λ_list)")
for i in eachindex(lambda_vals_list)
    lambda_vals = lambda_vals_list[i]
    plot!(findall(x->x==lambda_vals[1], lambda_vals_list_flattened), lambda_vals, color = lambda_colors[i], marker=:circle, label="λ=$(lambda_vals[1])", leg=:bottomleft)
end        
plot!(ψ_vals_exact_list, c=:red, linestyle=:dash, label="ψ(x)_exact", leg=:bottomleft)
hline!([ϵ], label="ϵ=$ϵ", c=:grey)
savefig("$dir/$(pa.game_name)_λ_list=($λ_list)_homotopy_linear.png")
println("saved to '$dir/$(pa.game_name)_λ_list=($λ_list)_homotopy_linear.png'")
println("--------------------------------\n")
IJulia.display(p2)

In [ ]:
using JLD2

# instantiate a routing game (p, E, s) with desired Nash sol x̂
game_name, g, p, E, E_diag, s_reduced, x̂ = grid_graph3_4_players_reduced()

# assign parameters
α = 0.005
ϵ = 1e-3
ρ = 0.5
max_iter = 10
@show λ_list = [1.0/(2^i) for i in 1:8]

# create an individual folder under homotopy_results/
dir = "homotopy_results/$(game_name)"; mkpath(dir) # mkdir if not exists

In [ ]:
""" RUNNING EXP: PARAMETER CHOICE """
# calling method
println("----------- $(game_name)_ρ=($ρ)_λ_list=($λ_list)_α=($α)_ϵ=($ϵ) -----------")
x, b, C, ψ_vals_list, violation_metrics_list, lambda_vals_list, v, ∇̂ψ_C_norm_list, D_norm_list, J_norm_list, pinv_J_norm_list, F_norm_list = homotopy_exp_parameter_choice(p, E, E_diag, s_reduced, x̂, λ_list, α, ϵ, ρ, max_iter)

# save data to folder
println("saving result to '$dir/$(game_name)_λ_list=($λ_list)_homotopy.jld2'")
@save "$dir/$(game_name)_λ_list=($λ_list)_homotopy.jld2" ρ λ_list α ϵ max_iter lambda_vals_list ψ_vals_list violation_metrics_list v ∇̂ψ_C_norm_list D_norm_list J_norm_list pinv_J_norm_list F_norm_list
println("--------------------------------")

@show findall(x->x>=0.2, x[1:24])   # expected [9, 13]

In [ ]:
using Plots, IJulia
mkpath("current_homotopy")

@load "$dir/$(game_name)_λ_list=($λ_list)_homotopy.jld2" ρ λ_list α ϵ max_iter lambda_vals_list ψ_vals_list violation_metrics_list v ∇̂ψ_C_norm_list D_norm_list J_norm_list pinv_J_norm_list F_norm_list

# plot single axis (log-scaled)
p1 = plot(lambda_vals_list, yscale=:log10, c=:green, label="λ", leg=:bottomleft, xlabel="iter", ylabel="log10 scaled")
plot!(ψ_vals_list, yscale=:log10, c=:blue, linestyle=:dash, label="ψ(x)", leg=:bottomleft)
plot!(violation_metrics_list, yscale=:log10, c=:red, linestyle=:dash, label="violation metric", leg=:bottomleft)
hline!([1e-3], label="1e-3", c=:grey)
plot!(title="ρ=($ρ), α=($α), ϵ=($ϵ)")
println("saving plot to '$dir/$(game_name)_λ_list=($λ_list)_homotopy.png'")
savefig("$dir/$(game_name)_λ_list=($λ_list)_homotopy.png")
IJulia.display(p1)

# plot helper_metrics_plot
# plot!(pinv_J_norm_list, label="pinv_J_norm_list", yscale=:log10)
p2 = plot(F_norm_list, label="F_norm_list", yscale=:log10)
# plot!(∇̂ψ_C_norm_list, label="∇̂ψ_C_norm_list", yscale=:log10, leg=:bottomleft)
# plot!(D_norm_list, label="D_norm_list", yscale=:log10)
# plot!(J_norm_list, label="J_norm_list", yscale=:log10)
hline!([1e-3], label="1e-3", c=:grey)
println("saving plot to '$dir/$(game_name)_λ_list=($λ_list)_helper_metrics.png'")
savefig("$dir/$(game_name)_λ_list=($λ_list)_helper_metrics.png")
IJulia.display(p2)